In [2]:
import os
import pandas as pd
import numpy as np
import json

In [ ]:
sf_police_budget_data = pd.read_csv('./sanfranciscodataset/plot_data/total_police_spending.csv')

In [ ]:
sf_police_budget_data = sf_police_budget_data.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:

with open('sf_police_budget_data.json', 'w') as f:
    json.dump(sf_police_budget_data, f)

In [ ]:
#SF incidents data
incidents = pd.read_csv('./sanfranciscodataset/plot_data/yearly_reported_incidents.csv')

In [ ]:
sf_incidents_json = incidents.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:
with open('sf_incidents.json', 'w') as f:
    json.dump(sf_incidents_json, f)

In [ ]:
#SF UOF
sf_force = pd.read_csv('./sanfranciscodataset/plot_data/sf_police_uof.csv')

In [ ]:
sf_force_json = sf_force.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')

In [2]:
#Spending data
df = pd.read_csv('Spending_Revenue_cleaned.csv', sep=',')

df_police = df[df['Department'] == 'POL Police']
years = df['Fiscal Year'].unique()
years
df_year = []
for index, year in enumerate(df_police['Fiscal Year'].unique()):
    df_year.append( df[df['Fiscal Year'] == year])
    
program_sums = []
for i in range(19):
    program_sums.append( pd.DataFrame(df_year[i].groupby('Program').Amount.agg('sum')) )
    col_year = years[i]
    program_sums[i].columns = [col_year]

important_programs = ['Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs=pd.concat(program_sums, axis=1)
df_agg_programs.fillna(0, inplace=True)
df_agg_programs = df_agg_programs.loc[important_programs]
df_agg_programs = df_agg_programs.T
df_agg_programs = df_agg_programs.reset_index()
df_agg_programs.columns = ['Year', 'Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs

,Year,Patrol,Operations And Administration,Special Operations,SFPD-Recruitment And Examination Program,SFPD Training
0,1999,1.566657e+08,4.110268e+07,151017.92,1292994.89,1490869.85
1,2000,1.763137e+08,5.329274e+07,305729.53,1544261.40,1380405.97
2,2001,1.867238e+08,5.817293e+07,163570.07,1735868.52,3000961.24
3,2002,1.947401e+08,5.272834e+07,291461.02,1344727.58,2465499.89
4,2003,1.904606e+08,4.906499e+07,9285482.67,1383969.78,1099537.23
5,2004,1.806452e+08,5.071372e+07,10083238.54,991255.91,1048983.57
6,2005,1.807123e+08,7.997473e+07,0.00,0.00,731208.16
7,2006,1.933768e+08,9.016354e+07,0.00,0.00,223558.12
8,2007,2.041555e+08,9.958305e+07,0.00,0.00,1261131.59
9,2008,2.310162e+08,1.036423e+08,0.00,0.00,855390.40


In [3]:
sf_police_budget_detail = df_agg_programs.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_police_budget_detail

'{"Year":1999,"Patrol":156665669.1800000072,"Operations And Administration":41102684.790000014,"Special Operations":151017.92,"SFPD-Recruitment And Examination Program":1292994.8900000004,"SFPD Training":1490869.8500000001}, {"Year":2000,"Patrol":176313685.9399999976,"Operations And Administration":53292740.0099999681,"Special Operations":305729.53,"SFPD-Recruitment And Examination Program":1544261.3999999999,"SFPD Training":1380405.97}, {"Year":2001,"Patrol":186723778.8900001049,"Operations And Administration":58172934.3600000069,"Special Operations":163570.07,"SFPD-Recruitment And Examination Program":1735868.5200000003,"SFPD Training":3000961.2400000002}, {"Year":2002,"Patrol":194740096.1700000465,"Operations And Administration":52728335.9700000063,"Special Operations":291461.02,"SFPD-Recruitment And Examination Program":1344727.5800000001,"SFPD Training":2465499.8900000001}, {"Year":2003,"Patrol":190460560.4300001264,"Operations And Administration":49064986.159999989,"Special Opera

In [4]:
with open('sf_police_budget_detail.json', 'w') as f:
    json.dump(sf_police_budget_detail, f)

In [3]:
#Population Share
pop_2010 = pd.read_csv('Intercensal_2000-2010_Total_Age-Race.xls - Total.csv', parse_dates=True)
pop_2020 = pd.read_csv('P3-Excel-Orange-YubaCos.xlsx - Orange to Yuba.csv')

In [4]:
pop_2010_long = pop_2010.melt(id_vars=['CountyCode', 'CountyName', 'RaceCode', 'RaceName', 'Gender', 'Age'], var_name='year', value_name='population')
pop_2010_long = pop_2010_long.loc[(pop_2010_long['Gender'] == 'Total') & (pop_2010_long['CountyCode'] <= 58), ['CountyName', 'RaceName', 'year', 'population']]
pop_2010_long

,CountyName,RaceName,year,population
0,Alameda,White,4/1/2000,5575
1,Alameda,White,4/1/2000,5550
2,Alameda,White,4/1/2000,5573
3,Alameda,White,4/1/2000,5632
4,Alameda,White,4/1/2000,5884
...,...,...,...,...
499844,Yuba,Multi-Race,4/1/2010,0
499845,Yuba,Multi-Race,4/1/2010,0
499846,Yuba,Multi-Race,4/1/2010,1
499847,Yuba,Multi-Race,4/1/2010,0


In [5]:
pop_2010_long['year'] = pd.to_datetime(pop_2010_long.year)
pop_2010_long.year = pop_2010_long.year.dt.year
pop_2010_long = pop_2010_long.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2010_long

,CountyName,RaceName,year,population
0,Alameda,American Indian,2000,10808
1,Alameda,American Indian,2001,5145
2,Alameda,American Indian,2002,4937
3,Alameda,American Indian,2003,4774
4,Alameda,American Indian,2004,4633
...,...,...,...,...
4461,Yuba,White,2006,42714
4462,Yuba,White,2007,42991
4463,Yuba,White,2008,43201
4464,Yuba,White,2009,42833


In [6]:
pop_2010_long.RaceName.unique()

array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [7]:
pop_2020 = pop_2020[['County Name', 'Race Name', 'year', 'Total Pop']]
pop_2020.columns = ['CountyName', 'RaceName', 'year', 'population']
pop_2020.CountyName = pop_2020.CountyName.str.replace(' County', '')

In [8]:
pop_2020 = pop_2020.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2020

,CountyName,RaceName,year,population
0,Orange,"American Indian or Alaska Native, Non-Hispanic",2010,6316
1,Orange,"American Indian or Alaska Native, Non-Hispanic",2011,6502
2,Orange,"American Indian or Alaska Native, Non-Hispanic",2012,6684
3,Orange,"American Indian or Alaska Native, Non-Hispanic",2013,6871
4,Orange,"American Indian or Alaska Native, Non-Hispanic",2014,7151
...,...,...,...,...
10348,Yuba,"White, Non-Hispanic",2056,47069
10349,Yuba,"White, Non-Hispanic",2057,47121
10350,Yuba,"White, Non-Hispanic",2058,47296
10351,Yuba,"White, Non-Hispanic",2059,47454


In [9]:
pop_2020.RaceName = pop_2020.RaceName.str.replace(', Non-Hispanic', '')
pop_2020.RaceName = pop_2020.RaceName.str.replace('American Indian or Alaska Native', 'American Indian')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Hispanic \(any race\)', 'Hispanic')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Native Hawaiian or Pacific Islander', 'Pacific Islander')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Multiracial \(two or more of above races\)', 'Multi-Race')
pop_2020.RaceName.unique()

<ipython-input-9-b598783ce459>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  pop_2020.RaceName = pop_2020.RaceName.str.replace('Hispanic \(any race\)', 'Hispanic')
<ipython-input-9-b598783ce459>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  pop_2020.RaceName = pop_2020.RaceName.str.replace('Multiracial \(two or more of above races\)', 'Multi-Race')


array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [10]:
pop_data_all_years = pop_2010_long.append(pop_2020)

In [11]:
pop_data_all_years.RaceName.unique()

array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [12]:
sf_pop_data = pop_data_all_years.loc[(pop_data_all_years['CountyName'] == 'San Francisco') & (pop_data_all_years['year'] <= 2021)]

In [13]:
sf_pop_data.to_csv('sf_pop_data_by_race.csv')

In [14]:
sf_pop_data['total_pop'] = sf_pop_data.groupby(['CountyName', 'year']).population.transform('sum')
sf_pop_data['pop_share'] = (sf_pop_data['population'] / sf_pop_data['total_pop']) * 100


<ipython-input-14-a5527a49e1c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_pop_data['total_pop'] = sf_pop_data.groupby(['CountyName', 'year']).population.transform('sum')
<ipython-input-14-a5527a49e1c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_pop_data['pop_share'] = (sf_pop_data['population'] / sf_pop_data['total_pop']) * 100


In [38]:
sf_pop_data

,CountyName,RaceName,year,population,total_pop,pop_share
2856,San Francisco,American Indian,2010,1925,810369,0.237546
2857,San Francisco,American Indian,2011,2064,821235,0.251329
2858,San Francisco,American Indian,2012,2218,834572,0.265765
2859,San Francisco,American Indian,2013,2375,845063,0.281044
2860,San Francisco,American Indian,2014,2531,854587,0.296166
...,...,...,...,...,...,...
3169,San Francisco,White,2017,366890,880343,41.675801
3170,San Francisco,White,2018,366750,886618,41.365052
3171,San Francisco,White,2019,366629,889360,41.223914
3172,San Francisco,White,2020,366690,892280,41.095844


In [15]:
uof = pd.read_csv('subject_uof_race_by_year.csv')

In [37]:
uof


,year,race,count
0,2015,American Indian,1
1,2015,Asian or Pacific Islander,6
2,2015,Black,27
3,2015,Hispanic,17
4,2015,Unknown,1
5,2015,White,24
6,2015,NaN,699
7,2016,American Indian,1
8,2016,Asian or Pacific Islander,139
9,2016,Black,786


In [ ]:
uof = uof.drop('Unnamed: 0', axis=1)

In [19]:
uof['race'].unique()


array(['American Indian', 'Asian or Pacific Islander', 'Black',
       'Hispanic', 'Unknown', 'White', nan], dtype=object)

In [20]:
sf_pop_data.RaceName.unique()
sf_pop_data.RaceName = sf_pop_data.RaceName.str.replace('Asian', 'Asian or Pacific Islander')
sf_pop_data.RaceName = sf_pop_data.RaceName.str.replace('Pacific Islander', 'Asian or Pacific Islander')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
sf_pop_data.RaceName.unique()

array(['American Indian', 'Asian or Asian or Pacific Islander', 'Black',
       'Hispanic', 'Multi-Race', 'Asian or Pacific Islander', 'White'],
      dtype=object)

In [51]:
sf_uof_by_race = sf_pop_data.merge(uof, left_on=['year', 'RaceName'], right_on=['year', 'race'])

In [52]:
sf_uof_by_race['rate_per_100k'] = (sf_uof_by_race['count'] / sf_uof_by_race['population']) * 100000

In [53]:
sf_uof_by_race = sf_uof_by_race[['year', 'race', 'rate_per_100k']].pivot(index='year', columns=['race'], values='rate_per_100k').reset_index()

In [54]:
sf_uof_by_race.columns

Index(['year', 'American Indian', 'Asian or Pacific Islander', 'Black',
       'Hispanic', 'White'],
      dtype='object', name='race')

In [55]:
sf_uof_by_race

race,year,American Indian,Asian or Pacific Islander,Black,Hispanic,White
0,2015,36.791759,228.136882,55.858987,13.131469,6.606529
1,2016,34.734283,5423.332033,1627.026020,300.330287,112.590401
2,2017,NaN,3485.576923,1387.357780,276.338250,107.934258
3,2018,NaN,3507.306889,1273.648299,223.745995,113.156101
4,2019,57.487784,2400.362319,978.193146,187.991384,79.099035
5,2020,28.089888,733.137830,304.363235,77.705405,28.634541


In [56]:
sf_uof_by_race_json = sf_uof_by_race.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_uof_by_race_json

'{"year":2015,"American Indian":36.7917586461,"Asian or Pacific Islander":228.1368821293,"Black":55.8589870904,"Hispanic":13.1314691797,"White":6.6065289022}, {"year":2016,"American Indian":34.7342827371,"Asian or Pacific Islander":5423.3320327741,"Black":1627.0260199963,"Hispanic":300.3302865043,"White":112.5904010519}, {"year":2017,"American Indian":null,"Asian or Pacific Islander":3485.5769230769,"Black":1387.3577802276,"Hispanic":276.338250266,"White":107.9342582245}, {"year":2018,"American Indian":null,"Asian or Pacific Islander":3507.3068893528,"Black":1273.6482993901,"Hispanic":223.7459950228,"White":113.1561008862}, {"year":2019,"American Indian":57.4877838459,"Asian or Pacific Islander":2400.3623188406,"Black":978.1931464174,"Hispanic":187.9913843625,"White":79.099034719}, {"year":2020,"American Indian":28.0898876404,"Asian or Pacific Islander":733.137829912,"Black":304.363234589,"Hispanic":77.7054050966,"White":28.6345414383}'

In [58]:
with open('sf_race_uof_and_pop_share.json', 'w') as f:
    json.dump(sf_uof_by_race_json, f)